<a href="https://colab.research.google.com/github/emielrobben/Capstone_AUC/blob/master/AI_TTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq ipdb
import ipdb

In [ ]:
%pdb off

Automatic pdb calling has been turned OFF


In [ ]:
import numpy as np
import random
import time
from math import sqrt, log
from copy import deepcopy
from IPython.display import clear_output
from google.colab import output

class MCTSNode:
    def __init__(self, state, symbol,parent=None):
        self.state = state  # Tic-Tac-Toe board state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0
        self.untried_moves = self.get_legal_moves()
        self.symbol = symbol

    def get_legal_moves(self):
        return [(i, j) for i in range(len(self.state)) for j in range(len(self.state[0])) if self.state[i, j] == ""]

    def best_child(self, exploration_weight=1.41):
        return max(self.children, key=lambda child: child.value / (child.visits + 1e-6) + exploration_weight * sqrt(log(self.visits + 1) / (child.visits + 1e-6)))

    def expand(self):
        if not self.untried_moves:
            return self  # Return self if no moves left to expand
        move = self.untried_moves.pop()
        #new_state = self.state.copy()
        new_state = deepcopy(self.state)
        #new_state[move] = "X" if np.count_nonzero(self.state == "X") <= np.count_nonzero(self.state == "O") else "O"
        new_state[move] = self.symbol
        child_node = MCTSNode(state=new_state,symbol=self.symbol,parent=self)
        self.children.append(child_node)
        return child_node

    def update(self, result):
        self.visits += 1
        self.value += result


class MCTS:
    def __init__(self, symbol, time_limit=5.0, max_iterations=1000):
        self.time_limit = time_limit
        self.max_iterations = max_iterations
        self.symbol = symbol

    def search(self, root):
        start_time = time.time()
        iterations = 0

        #if root.untried_moves:
        #    root = root.expand()  # Ensure root expands at least once

        #while time.time() - start_time < self.time_limit and iterations < self.max_iterations:
        node = self.select(root)
        #if node.untried_moves:
        node = node.expand()
        result = self.simulate(node.state)
        self.backpropagate(node, result)
        iterations += 1

        #return max(root.children, key=lambda c: c.visits, default=root)  # Choose best move by visits
        return root.best_child()  # Choose best move by value

    def select(self, node):
        # here we use a tree policy for selection
        while node.children and not node.untried_moves:
            node = node.best_child()
        return node

    def check_winner(self, state):
     for player in ["X", "O"]:
      for i in range(len(state)):
          if all(state[i, :] == player) or all(state[:, i] == player):
              return player
      if all(state.diagonal() == player) or all(np.fliplr(state).diagonal() == player):
          return player
     return None

    def simulate(self, state):
        state=deepcopy(state)
        #current_player = "X" if np.count_nonzero(state == "X") <= np.count_nonzero(state == "O") else "O"
        current_player = self.symbol
        while True:
            legal_moves = [(i, j) for i in range(len(state)) for j in range(len(state[0])) if state[i, j] == ""]
            if not legal_moves:
                return 0  # Draw
            move = random.choice(legal_moves)
            state[move] = current_player
            winner=self.check_winner(state)
            if winner==current_player:
                return 1 if current_player == "X" else -1
            current_player = "O" if current_player == "X" else "X"

    def backpropagate(self, node, result):
        while node:
            node.update(result)
            result = -result  # Alternate win/loss perspective
            node = node.parent

class TicTacToe:
    def __init__(self,human_player = "X",current_player="X",board_shape=3):
        self.state = np.full((board_shape, board_shape), "")
        self.symbols=["X","O"]
        if human_player in self.symbols:
            self.human_player = human_player
            self.ai_player2 = list(set(self.symbols) - {human_player})[0]
        else:
         self.ai_player = "X"
         self.ai_mcts = MCTS(self.ai_player)
         self.ai_player2= "O"
         self.human_player = None

        self.current_player = current_player
        self.ai2_mcts = MCTS(self.ai_player2)

    def display_board(self):
        size = len(self.state)

        # Print column numbers
        print("  " + " ".join(str(i) for i in range(size)))

        for i in range(size):
            # Print row number and row content
            print(f"{i} " + "|".join(self.state[i]))

            # Print row separators (except after the last row)
            if i < size - 1:
                print("  " + "-+" * (size - 1) + "-")

        print()

    def get_human_move(self):
     while True:
      try:
          row = int(input(f"Enter row (0-{len(self.state)-1}): "))
          col = int(input(f"Enter column (0-{len(self.state[0])-1}): "))
          if self.state[row, col] == "":
              self.state[row, col] = self.human_player
              self.current_player = self.ai_player2  # Switch turn
              break
          else:
              print("Invalid move, try again!")
      except (ValueError, IndexError):
          print("Invalid input!")

    def get_ai_move(self,mcts):
     # AI's turn (MCTS)
     print(f"AI({mcts.symbol}) is thinking...")
     #time.sleep(1)
     root = MCTSNode(state=deepcopy(self.state),symbol=mcts.symbol)
     best_move = mcts.search(root)
     self.state = best_move.state
     self.current_player = list(set(self.symbols) - {mcts.symbol})[0]

    def play_game(self):
        while True:
            self.display_board()

            def player_one_ai_player():
              self.get_ai_move(self.ai_mcts)

            player_one=self.human_player if self.human_player else self.ai_player
            player_one_move = self.get_human_move if self.human_player else player_one_ai_player

            if self.current_player == player_one:
              player_one_move()
            else:
              self.get_ai_move(self.ai2_mcts)

            winner=self.ai2_mcts.check_winner(self.state)
            if winner:
                self.display_board()
                print(f"{winner} wins!")
                break
            elif "" not in self.state:
                self.display_board()
                print("It's a draw!")
                break

if __name__ == "__main__":
    board_shape = input("Enter board shape (default is 3x3): ")
    if board_shape:
        board_shape = int(board_shape)
    human_player = input("Choose a player (X or O): ")
    game = TicTacToe(human_player=human_player,board_shape=board_shape)
    game.play_game()


Enter board shape (default is 3x3): 4
Choose a player (X or O): s
  0 1 2 3
0 |||
  -+-+-+-
1 |||
  -+-+-+-
2 |||
  -+-+-+-
3 |||

AI(X) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |||
  -+-+-+-
2 |||
  -+-+-+-
3 |||X

AI(O) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |||
  -+-+-+-
2 |||
  -+-+-+-
3 ||O|X

AI(X) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |||
  -+-+-+-
2 |||
  -+-+-+-
3 |X|O|X

AI(O) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |||
  -+-+-+-
2 |||
  -+-+-+-
3 O|X|O|X

AI(X) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |||
  -+-+-+-
2 |||X
  -+-+-+-
3 O|X|O|X

AI(O) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |||
  -+-+-+-
2 ||O|X
  -+-+-+-
3 O|X|O|X

AI(X) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |||
  -+-+-+-
2 |X|O|X
  -+-+-+-
3 O|X|O|X

AI(O) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |||
  -+-+-+-
2 O|X|O|X
  -+-+-+-
3 O|X|O|X

AI(X) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |||X
  -+-+-+-
2 O|X|O|X
  -+-+-+-
3 O|X|O|X

AI(O) is thinking...
  0 1 2 3
0 |||
  -+-+-+-
1 |

In [4]:
import numpy as np
import random
import time
from math import sqrt, log
from copy import deepcopy
from IPython.display import clear_output
from google.colab import output

class MCTSNode:
    def __init__(self, state, symbol, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0
        self.untried_moves = self.get_legal_moves()
        self.symbol = symbol

    def get_legal_moves(self):
        return [(i, j) for i in range(len(self.state)) for j in range(len(self.state[0])) if self.state[i, j] == ""]

    def heuristic_score(self, state):
        # Simple heuristic: prefer center and corners
        score = 0
        center = (1, 1)
        corners = [(0, 0), (0, 2), (2, 0), (2, 2)]
        if state[center] == self.symbol:
            score += 4  # Center has highest strategic value
        for corner in corners:
            if state[corner] == self.symbol:
                score += 2  # Corners have high strategic value
        return score

    def best_child(self, exploration_weight=1.41):
        # Incorporate heuristic score into node selection
        return max(self.children, key=lambda child: (child.value / (child.visits + 1e-6) +
                                                     exploration_weight * sqrt(log(self.visits + 1) / (child.visits + 1e-6)) +
                                                     0.1 * self.heuristic_score(child.state)))

    def expand(self):
        if not self.untried_moves:
            return self  # Return self if no moves left to expand
        move = self.untried_moves.pop()
        new_state = deepcopy(self.state)
        new_state[move] = self.symbol
        child_node = MCTSNode(state=new_state, symbol=self.symbol, parent=self)
        self.children.append(child_node)
        return child_node

    def update(self, result):
        self.visits += 1
        self.value += result

class MCTS:
    def __init__(self, symbol, time_limit=5.0, max_iterations=1000):
        self.time_limit = time_limit
        self.max_iterations = max_iterations
        self.symbol = symbol

    def search(self, root):
        start_time = time.time()
        iterations = 0

        while time.time() - start_time < self.time_limit and iterations < self.max_iterations:
            node = self.select(root)
            node = node.expand()
            result = self.simulate(node.state)
            self.backpropagate(node, result)
            iterations += 1

        return root.best_child()  # Choose best move by value

    def select(self, node):
        while node.children and not node.untried_moves:
            node = node.best_child()
        return node

    def check_winner(self, state):
        for player in ["X", "O"]:
            for i in range(len(state)):
                if all(state[i, :] == player) or all(state[:, i] == player):
                    return player
            if all(state.diagonal() == player) or all(np.fliplr(state).diagonal() == player):
                return player
        return None

    def simulate(self, state):
        state = deepcopy(state)
        current_player = self.symbol
        while True:
            legal_moves = [(i, j) for i in range(len(state)) for j in range(len(state[0])) if state[i, j] == ""]
            if not legal_moves:
                return 0  # Draw
            move = random.choice(legal_moves)
            state[move] = current_player
            winner = self.check_winner(state)
            if winner == current_player:
                return 1 if current_player == "X" else -1
            current_player = "O" if current_player == "X" else "X"

    def backpropagate(self, node, result):
        while node:
            node.update(result)
            result = -result  # Alternate win/loss perspective
            node = node.parent

class TicTacToe:
    def __init__(self, human_player="X", current_player="X", board_shape=3):
        self.state = np.full((board_shape, board_shape), "")
        self.symbols = ["X", "O"]
        if human_player in self.symbols:
            self.human_player = human_player
            self.ai_player2 = list(set(self.symbols) - {human_player})[0]
        else:
            self.ai_player = "X"
            self.ai_mcts = MCTS(self.ai_player)
            self.ai_player2 = "O"
            self.human_player = None

        self.current_player = current_player
        self.ai2_mcts = MCTS(self.ai_player2)

    def display_board(self):
        size = len(self.state)
        print("  " + " ".join(str(i) for i in range(size)))
        for i in range(size):
            print(f"{i} " + "|".join(self.state[i]))
            if i < size - 1:
                print("  " + "-+" * (size - 1) + "-")
        print()

    def get_human_move(self):
        while True:
            try:
                row = int(input(f"Enter row (0-{len(self.state)-1}): "))
                col = int(input(f"Enter column (0-{len(self.state[0])-1}): "))
                if self.state[row, col] == "":
                    self.state[row, col] = self.human_player
                    self.current_player = self.ai_player2  # Switch turn
                    break
                else:
                    print("Invalid move, try again!")
            except (ValueError, IndexError):
                print("Invalid input!")

    def get_ai_move(self, mcts):
        print(f"AI({mcts.symbol}) is thinking...")
        root = MCTSNode(state=deepcopy(self.state), symbol=mcts.symbol)
        best_move = mcts.search(root)
        self.state = best_move.state
        self.current_player = list(set(self.symbols) - {mcts.symbol})[0]

    def play_game(self):
        while True:
            self.display_board()
            if self.current_player == self.human_player:
                self.get_human_move()
            else:
                self.get_ai_move(self.ai2_mcts)

            winner = self.ai2_mcts.check_winner(self.state)
            if winner:
                self.display_board()
                print(f"{winner} wins!")
                break
            elif "" not in self.state:
                self.display_board()
                print("It's a draw!")
                break

if __name__ == "__main__":
    board_shape = input("Enter board shape (default is 3x3): ")
    if board_shape:
        board_shape = int(board_shape)
    human_player = input("Choose a player (X or O): ")
    game = TicTacToe(human_player=human_player, board_shape=board_shape)
    game.play_game()


Enter board shape (default is 3x3): 3
Choose a player (X or O): O
  0 1 2
0 ||
  -+-+-
1 ||
  -+-+-
2 ||

AI(X) is thinking...
  0 1 2
0 X||
  -+-+-
1 ||
  -+-+-
2 ||

Enter row (0-2): 1
Enter column (0-2): 1
  0 1 2
0 X||
  -+-+-
1 |O|
  -+-+-
2 ||

AI(X) is thinking...
  0 1 2
0 X|X|
  -+-+-
1 |O|
  -+-+-
2 ||

Enter row (0-2): 0
Enter column (0-2): 2
  0 1 2
0 X|X|O
  -+-+-
1 |O|
  -+-+-
2 ||

AI(X) is thinking...
  0 1 2
0 X|X|O
  -+-+-
1 |O|
  -+-+-
2 ||X

Enter row (0-2): 2
Enter column (0-2): 0
  0 1 2
0 X|X|O
  -+-+-
1 |O|
  -+-+-
2 O||X

O wins!
